# A : Setup

##A.1 Authentication and Checking User Details.

In [107]:
# Authenticate to GCS.
from google.colab import auth
auth.authenticate_user()

In [108]:
# Getting the email account that was used to login to this session
import requests
gcloud_token = !gcloud auth print-access-token
gcloud_tokeninfo = requests.get('https://www.googleapis.com/oauth2/v3/tokeninfo?access_token=' + gcloud_token[0]).json()
print(gcloud_tokeninfo['email'])

cfemilioc@gmail.com


##A.2 Setup the Project Parameters.

In [116]:
# Change the following if you are not in the rsa-dev-10 project
# These parameters are generated outside of this notebook
# Project: rsa-mla-prod

# Project: rsa-dev-10
project_id = 'rsa-mono-site5'
bucket_id = 'rsa-mono-site5'
# ip_addr_name = 'rsa-static-west'
# project_id = 'mla-metric'
# bucket_id = 'mla-metric'
ip_addr_name = 'rsa-prod'

# These are parameters you setup for this deployment
region = 'us-central1'
zone = 'us-central1-f'


In [117]:
import os
os.environ['DEVSHELL_PROJECT_ID'] = project_id
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

!gcloud config set project {project_id} -q

Updated property [core/project].


In [118]:
!gcloud config set compute/zone {zone}
!gcloud config set compute/region {region}

Updated property [compute/zone].
Updated property [compute/region].


### Install Kubectl

In [119]:
!curl -LO https://storage.googleapis.com/kubernetes-release/release/`curl -s \
 https://storage.googleapis.com/kubernetes-release/release/stable.txt`/bin/linux/amd64/kubectl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.9M  100 41.9M    0     0   141M      0 --:--:-- --:--:-- --:--:--  141M


In [120]:
!chmod +x kubectl
!sudo mv ./kubectl /usr/local/bin/kubectl
!kubectl version --client

Client Version: version.Info{Major:"1", Minor:"18", GitVersion:"v1.18.6", GitCommit:"dff82dc0de47299ab66c83c626e08b245ab19037", GitTreeState:"clean", BuildDate:"2020-07-15T16:58:53Z", GoVersion:"go1.13.9", Compiler:"gc", Platform:"linux/amd64"}


##A.3 Enabling APIs

In [121]:
!gcloud services enable container.googleapis.com --async
!gcloud services enable containerregistry.googleapis.com --async
!gcloud services enable compute.googleapis.com --async
!gcloud services enable storage-api.googleapis.com --async
!gcloud services enable storage-component.googleapis.com --async                        
!gcloud services enable servicemanagement.googleapis.com --async
!gcloud services enable iam.googleapis.com --async
!gcloud services enable cloudbuild.googleapis.com --async

Asynchronous operation is in progress... Use the following command to wait for its completion:
 gcloud beta services operations wait operations/acf.03613c86-289c-4ff1-b8d5-b56b33c68b3d


# B : Git Repository Pull

In [122]:
%cd ~
!rm -rf ~/RSA-Mono-Site
!git clone https://github.com/cfphpflex/RSA-Mono-Site.git
%cd ~/RSA-Mono-Site
!./setup.sh

/root
Cloning into 'RSA-Mono-Site'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (145/145), done.
remote: Total 253 (delta 129), reused 204 (delta 83), pack-reused 0
Receiving objects: 100% (253/253), 1.27 MiB | 20.28 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/root/RSA-Mono-Site
Checking for required npm version...Completed.
Installing monolith dependencies...Completed.
Installing microservies dependencies...Completed.
Installing React app dependencies...Completed.
Building React app and placing into sub projects...Completed.

Script completed successfully!


In [123]:
!python main.py
#! nano main.py


 * Serving Flask app "main" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Traceback (most recent call last):
  File "main.py", line 29, in <module>
    app.run(host='0.0.0.0', port=8080, debug=True)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 1030, in run_simple
    s.bind(server_address)
OSError: [Errno 98] Address already in use


In [124]:
!ls -lrt

total 60
-rw-r--r-- 1 root root 3485 Jul 30 03:32 README.md
drwxr-xr-x 6 root root 4096 Jul 30 03:32 RSA-Core
-rwxr-xr-x 1 root root    9 Jul 30 03:32 index.html
-rwxr-xr-x 1 root root 2715 Jul 30 03:32 firestore.py
-rwxr-xr-x 1 root root 1336 Jul 30 03:32 deploy-monolith.sh
-rwxr-xr-x 1 root root 1049 Jul 30 03:32 app.yaml
-rwxr-xr-x 1 root root  705 Jul 30 03:32 main.py
-rwxr-xr-x 1 root root  196 Jul 30 03:32 requirements.txt
-rwxr-xr-x 1 root root 2814 Jul 30 03:32 storage.py
-rwxr-xr-x 1 root root 1312 Jul 30 03:32 setup.sh
drwxr-xr-x 3 root root 4096 Jul 30 03:32 microservices
drwxr-xr-x 2 root root 4096 Jul 30 03:33 logs
drwxr-xr-x 7 root root 4096 Jul 30 03:33 react-app
drwxr-xr-x 7 root root 4096 Jul 30 03:33 monolith
drwxr-xr-x 2 root root 4096 Jul 30 03:35 __pycache__


In [125]:
!gcloud container clusters create fancy-cluster --num-nodes 3

This will enable the autorepair feature for nodes. Please see https://cloud.google.com/kubernetes-engine/docs/node-auto-repair for more information on node autorepairs.
Created [https://container.googleapis.com/v1/projects/rsa-mono-site5/zones/us-central1-f/clusters/fancy-cluster].
To inspect the contents of your cluster, go to: https://console.cloud.google.com/kubernetes/workload_/gcloud/us-central1-f/fancy-cluster?project=rsa-mono-site5
kubeconfig entry generated for fancy-cluster.
NAME           LOCATION       MASTER_VERSION  MASTER_IP       MACHINE_TYPE   NODE_VERSION   NUM_NODES  STATUS
fancy-cluster  us-central1-f  1.15.12-gke.2   35.192.166.246  n1-standard-1  1.15.12-gke.2  3          RUNNING


In [126]:
!gcloud config set container/use_application_default_credentials true

Updated property [container/use_application_default_credentials].


In [127]:
!gcloud compute instances list

NAME                                          ZONE           MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
gke-fancy-cluster-default-pool-93ff2869-7j6t  us-central1-f  n1-standard-1               10.128.0.4   35.238.58.195  RUNNING
gke-fancy-cluster-default-pool-93ff2869-ddh5  us-central1-f  n1-standard-1               10.128.0.2   34.72.165.16   RUNNING
gke-fancy-cluster-default-pool-93ff2869-z89z  us-central1-f  n1-standard-1               10.128.0.3   35.202.62.224  RUNNING


Get Credentials

In [128]:
!gcloud config set container/use_application_default_credentials true
!gcloud container clusters get-credentials fancy-cluster
!kubectl cluster-info

Updated property [container/use_application_default_credentials].
Fetching cluster endpoint and auth data.
kubeconfig entry generated for fancy-cluster.
Kubernetes master is running at https://35.192.166.246
GLBCDefaultBackend is running at https://35.192.166.246/api/v1/namespaces/kube-system/services/default-http-backend:http/proxy
Heapster is running at https://35.192.166.246/api/v1/namespaces/kube-system/services/heapster/proxy
KubeDNS is running at https://35.192.166.246/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy
Metrics-server is running at https://35.192.166.246/api/v1/namespaces/kube-system/services/https:metrics-server:/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [129]:
!pwd
!python main.py
!./deploy-monolith.sh





/root/RSA-Mono-Site
 * Serving Flask app "main" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
Traceback (most recent call last):
  File "main.py", line 29, in <module>
    app.run(host='0.0.0.0', port=8080, debug=True)
  File "/usr/local/lib/python3.6/dist-packages/flask/app.py", line 990, in run
    run_simple(host, port, self, **options)
  File "/usr/local/lib/python3.6/dist-packages/werkzeug/serving.py", line 1030, in run_simple
    s.bind(server_address)
OSError: [Errno 98] Address already in use
Enabling Cloud Build APIs...Completed.
Building Monolith Container...Completed.
Deploying Monolith To GKE Cluster...Completed.

Please run the following command to find the IP address for the monolith service:  kubectl get service monolith

Deployment Complete


In [131]:
!gcloud compute instances list
!kubectl get service

NAME                                          ZONE           MACHINE_TYPE   PREEMPTIBLE  INTERNAL_IP  EXTERNAL_IP    STATUS
gke-fancy-cluster-default-pool-93ff2869-7j6t  us-central1-f  n1-standard-1               10.128.0.4   35.238.58.195  RUNNING
gke-fancy-cluster-default-pool-93ff2869-ddh5  us-central1-f  n1-standard-1               10.128.0.2   34.72.165.16   RUNNING
gke-fancy-cluster-default-pool-93ff2869-z89z  us-central1-f  n1-standard-1               10.128.0.3   35.202.62.224  RUNNING
NAME         TYPE           CLUSTER-IP     EXTERNAL-IP   PORT(S)        AGE
kubernetes   ClusterIP      10.19.240.1    <none>        443/TCP        4m50s
monolith     LoadBalancer   10.19.244.82   34.66.54.67   80:30243/TCP   72s


# C: Build Container Image

##Build Order Container Image and Deploy It

In [ ]:
%cd ~/RSA-Mono-Site/microservices/src/orders
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/orders:1.0.0 .

In [150]:
!kubectl create deployment orders --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/orders:1.0.0

Error from server (AlreadyExists): deployments.apps "orders" already exists


In [166]:
!kubectl expose deployment orders --type=LoadBalancer --port 80 --target-port 8080

Error from server (AlreadyExists): services "orders" already exists


##Build Product Container Image and Deploy It

*italicized text*##Build products Container Image and Deploy It

In [ ]:
%cd ~/RSA-Mono-Site/microservices/src/products
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0 .

In [133]:
!kubectl create deployment products --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0

deployment.apps/products created


In [165]:
!kubectl expose deployment products --type=LoadBalancer --port 80 --target-port 8080

Error from server (AlreadyExists): services "products" already exists


Build USERS Container Image and Deploy It

In [ ]:
%cd ~/RSA-Mono-Site/microservices/src/users
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/users:1.0.0 .

In [ ]:
!kubectl create deployment users --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/users:1.0.0

In [ ]:
!kubectl expose deployment users --type=LoadBalancer --port 80 --target-port 8082

In [ ]:
##Build FrontEnd  Container Image and Deploy It

In [ ]:
%cd ~/RSA-Mono-Site/microservices/src/frontend
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0 .

In [ ]:
!kubectl create deployment frontend --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0

In [146]:
!kubectl expose deployment frontend --type=LoadBalancer --port 80 --target-port 8082

service/frontend exposed


## Build Frontend Container and Deploy It

In [154]:
!kubectl get service

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
frontend     LoadBalancer   10.19.241.50    34.69.153.194   80:30145/TCP   2m39s
kubernetes   ClusterIP      10.19.240.1     <none>          443/TCP        17m
monolith     LoadBalancer   10.19.244.82    34.66.54.67     80:30243/TCP   13m
orders       LoadBalancer   10.19.248.36    34.69.61.52     80:32633/TCP   37s
products     LoadBalancer   10.19.244.193   35.224.86.104   80:30492/TCP   10m
users        LoadBalancer   10.19.251.127   35.238.211.15   80:30592/TCP   8m52s


### Copy the IP address of order and product LoadBalancer and paste it below

In [ ]:
ORDER_SERVICE_IP_ARR =!kubectl get -o jsonpath="34.69.61.52" --namespace=default services orders
ORDER_SERVICE_IP = ORDER_SERVICE_IP_ARR[0]
print(ORDER_SERVICE_IP)
PROD_SERVICE_IP_ARR =!kubectl get -o jsonpath="35.224.86.104" --namespace=default services products
PROD_SERVICE_IP = PROD_SERVICE_IP_ARR[0]
print(PROD_SERVICE_IP)

In [156]:
%cd ~/RSA-Mono-Site/react-app
!cp ./.env.monolith ./.env
!sed -i 's+/service/orders+http://{ORDER_SERVICE_IP}/api/orders+g' ./.env
!sed -i 's+/service/products+http://{PROD_SERVICE_IP}/api/products+g' ./.env
!cat ./.env

/root/RSA-Mono-Site/react-app
REACT_APP_ORDERS_URL=http://34.69.61.52/api/orders
REACT_APP_PRODUCTS_URL=http://35.224.86.104/api/products

In [ ]:
!npm run build

In [ ]:
%cd ~/RSA-Mono-Site/microservices/src/frontend
!gcloud builds submit --tag gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0 .

In [ ]:
!kubectl create deployment frontend --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/frontend:1.0.0

In [ ]:
!kubectl expose deployment frontend --type=LoadBalancer --port 80 --target-port 8080

## How can I Get the External-IP address?

In [161]:
!kubectl get nodes

NAME                                           STATUS   ROLES    AGE   VERSION
gke-fancy-cluster-default-pool-93ff2869-7j6t   Ready    <none>   19m   v1.15.12-gke.2
gke-fancy-cluster-default-pool-93ff2869-ddh5   Ready    <none>   19m   v1.15.12-gke.2
gke-fancy-cluster-default-pool-93ff2869-z89z   Ready    <none>   19m   v1.15.12-gke.2


In [162]:
!kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
frontend-569fd74db5-gw75v   1/1     Running   0          6m40s
monolith-5f947c777-jcw77    1/1     Running   0          16m
orders-7ccc9bb5bb-5vgqt     1/1     Running   0          3m43s
products-5d785f76b9-xhpgn   1/1     Running   0          12m
users-97589c765-qzgdn       1/1     Running   0          11m


In [163]:
!kubectl get service

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP     PORT(S)        AGE
frontend     LoadBalancer   10.19.241.50    34.69.153.194   80:30145/TCP   5m14s
kubernetes   ClusterIP      10.19.240.1     <none>          443/TCP        19m
monolith     LoadBalancer   10.19.244.82    34.66.54.67     80:30243/TCP   16m
orders       LoadBalancer   10.19.248.36    34.69.61.52     80:32633/TCP   3m12s
products     LoadBalancer   10.19.244.193   35.224.86.104   80:30492/TCP   12m
users        LoadBalancer   10.19.251.127   35.238.211.15   80:30592/TCP   11m


In [164]:
FRONT_SERVICE_IP_ARR=!kubectl get -o jsonpath="35.226.95.49" --namespace=default services frontend
FRONT_SERVICE_IP = FRONT_SERVICE_IP_ARR[0]
'http://'+FRONT_SERVICE_IP

'http://35.226.95.49'

## Resilency Test

In [102]:
!kubectl get pods

NAME                        READY   STATUS    RESTARTS   AGE
frontend-84c5597b4d-6wm9l   1/1     Running   0          20m
monolith-69bb74676d-tj8hr   1/1     Running   0          32m
orders-6bbc98c9c9-8s52n     1/1     Running   0          28m
products-68699cb54d-7nl5x   1/1     Running   0          27m
users-6d98786b78-dqrq7      1/1     Running   0          116s


In [103]:
!kubectl delete pod products-6fd4b78888-mqfwv


Error from server (NotFound): pods "products-6fd4b78888-mqfwv" not found


In [ ]:
!kubectl delete deployment products
#!kubectl delete

In [ ]:
!kubectl create deployment products --image=gcr.io/${GOOGLE_CLOUD_PROJECT}/products:1.0.0

In [ ]:
!curl {PROD_SERVICE_IP}/api/products

## Internal Name space

# C : Cleanup Project

###The code below will cause an error. It will prevent you from deleting the cluster unintentionally. <br>
###Execute the cluster delete when you are done to clean up.

In [ ]:
raise

In [ ]:
!gcloud container clusters list

In [ ]:
!gcloud container clusters delete fancy-cluster
